In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device="cuda" if torch.cuda.is_available() else "cpu"
print(device)

#hyper parameters
block_size=8
batch_size=4
vocab_size=50
max_iters=10000
learning_rate=3e-4

cuda


In [3]:
with open("wizard_of_oz.txt", 'r', encoding="utf-8") as f:
    text=f.read()
# creating an array of all the characters in text file
chars=sorted(set(text))   
print(chars)
print(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


# tokenizing the character

In [4]:
string_to_int={ch:i for i,ch in enumerate(chars)}
int_to_string={i:ch for i,ch in enumerate(chars)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda l:"".join([int_to_string[i] for i in l])

In [5]:
#to encode a word we use the following 
print(encode("hellow"))
encoded_word=encode("hellow")
decode_word=decode(encoded_word)
print(f"the encoded word is: {encoded_word}")
print(f"the decoded word is: {decode_word}")

[61, 58, 65, 65, 68, 76]
the encoded word is: [61, 58, 65, 65, 68, 76]
the decoded word is: hellow


In [6]:
# encoding text data to a tourch tensor
data=torch.tensor(encode(text), dtype=torch.long)
print(data[:50])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25])


In [27]:
#spliting data into train and val set
n=int(0.8 * len(data))

train_data=data[:n]
val_data=data[n:]

def get_batch(split):
    data=train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y=get_batch("train")
print(f"input: {x} ... output: \n {y}")


tensor([ 57679,  81753, 179339,  21257])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [11]:
x=train_data[:block_size]
y=train_data[1:block_size+1]

for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when the input is: {context} target is {target}")

when the input is: tensor([80]) target is 28
when the input is: tensor([80, 28]) target is 39
when the input is: tensor([80, 28, 39]) target is 42
when the input is: tensor([80, 28, 39, 42]) target is 39
when the input is: tensor([80, 28, 39, 42, 39]) target is 44
when the input is: tensor([80, 28, 39, 42, 39, 44]) target is 32
when the input is: tensor([80, 28, 39, 42, 39, 44, 32]) target is 49
when the input is: tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is 1


In [35]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #defining an embedding vector based on the vacab_size
        self.token_embeddeing_table=nn.Embedding(vocab_size, vocab_size)

    """ 
    below fucntion is used to compute the forward pass of the model and 
    compute the loss for a particular 
    """
    def forward(self, index, targets=None):
        #accessing the embedding vector for a paticular index token 
        logits = self.token_embeddeing_table(index)

        if targets is None:
            loss=None
        else:
            #we unpack the dimension using the .shape
            batch, time, channel=logits.shape
            #we pack the dimension using the .view, we flattenr the batch and time into single dimension to compute loss 
            logits=logits.view(batch*time, channel)
            # we flattern the target vector to get loss
            targets=targets.view(batch*time)
            loss=F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # the index form is (batch, time ) in refrance to the current context
        for _ in range(max_new_tokens):
            #getting the pridition 
            logits, loss=self.forward(index)
            
            #getting the last time step
            logits=logits[:, -1, :] # to make it (batch,  channel)

            #applying softmax to the probablity distribution
            prob=F.softmax(logits, dim=-1) 

            #getting a sample from the distribution
            index_next= torch.multinomial(prob, num_samples=1)

            #appending the generated token to the preexisiting data
            index=torch.cat((index, index_next), dim=-1)
        return index
    

model=BigramLanguageModel(vocab_size=50)
m=model.to(device)

context=torch.zeros((1,1), dtype=torch.long, device=device)
generated_characters=decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_characters)


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [26]:
# creating a pytorch optimizer
optimizer=torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):

    #sampling a batch data
    xb, yb=get_batch("train")

    #evaluating the loss
    logits, loss=model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.itme())

tensor([105519, 161897, 140883,   7986])


RuntimeError: shape '[32]' is invalid for input of size 28